<a href="https://colab.research.google.com/github/MarioTiara/COVID-19/blob/master/VGG16_120_epochs_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/MarioTiara/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 270 (delta 0), reused 0 (delta 0), pack-reused 265
Receiving objects: 100% (270/270), 106.16 MiB | 11.66 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Checking out files: 100% (648/648), done.


In [0]:
import os
os.chdir('/content/COVID-19/')

In [0]:
import glob
import numpy as np
import pandas as pd
import math
import seaborn as sns
import umap
import sys
import random
import warnings
import cv2
import os
import shutil
import itertools
import imutils
from matplotlib import pyplot
import matplotlib.pyplot as plt
from matplotlib import pyplot
from numpy import expand_dims
from PIL import Image
from scipy import misc
from os import listdir
from os.path import isfile,join
from collections import Counter
from random import shuffle
from collections import Counter
from itertools import chain
from tqdm import tqdm

In [0]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [0]:
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

In [0]:
import tensorflow as tf

In [0]:
RANDOM_SEED = 123

In [0]:
def load_data(dir_path, img_size=(100,100)):
  
    #Load resized images as np.arrays to workspace
    
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels

In [0]:
TRAIN_DIR = '/content/COVID-19/DATA_X_RAY/TRAIN_CROP/'
TEST_DIR = '/content/COVID-19/DATA_X_RAY/TEST_CROP/'
VAL_DIR = '/content/COVID-19/DATA_X_RAY/VAL_CROP/'
IMG_SIZE = (224,224)

In [0]:
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

  0%|          | 0/2 [00:00<?, ?it/s]

142 images loaded from /content/COVID-19/DATA_X_RAY/TRAIN_CROP/ directory.


  0%|          | 0/2 [00:00<?, ?it/s]

20 images loaded from /content/COVID-19/DATA_X_RAY/TEST_CROP/ directory.


100%|██████████| 2/2 [00:01<00:00,  1.21it/s]

40 images loaded from /content/COVID-19/DATA_X_RAY/VAL_CROP/ directory.


In [0]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(validation_labels)

In [0]:
def preprocess_imgs(set_name, img_size):
    """
    Resize and apply VGG-15 preprocessing
    """
    set_new = []
    for img in set_name:
        img = cv2.resize(
            img,
            dsize=img_size,
            interpolation=cv2.INTER_CUBIC
        )
        set_new.append(tf.keras.applications.vgg16.preprocess_input(img))
    return np.array(set_new)

In [0]:
X_train_prep = preprocess_imgs(set_name=X_train, img_size=IMG_SIZE)
X_test_prep = preprocess_imgs(set_name=X_test, img_size=IMG_SIZE)
X_val_prep = preprocess_imgs(set_name=X_val, img_size=IMG_SIZE)

In [0]:
IMG_SHAPE= (224,224,3)
vgg = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

In [0]:
output = vgg.layers[-1].output
output =tf.keras.layers.Flatten()(output)
vgg_model = tf.keras.models.Model(inputs=vgg.input, outputs=output)

In [0]:
vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False

In [0]:
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])   

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fb89e4e56a0>,input_8,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb8b2113d68>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb89e4e8d68>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb8b2151e48>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb89e4756a0>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb89e475a58>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fb89e47b048>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb89e47ba90>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb89e480da0>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb89e487470>,block3_conv3,False


In [0]:
model =tf.keras.models.Sequential()
model.add(vgg_model)
model.add(tf.keras.layers.Dense(512, activation='relu', input_dim=input_shape))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_7 (Model)              (None, 25088)             14714688  
_________________________________________________________________
dense_18 (Dense)             (None, 512)               12845568  
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 513       
Total params: 27,823,425
Trainable params: 13,108,737
Non-trainable params: 14,714,688
________________________________